In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import TSMixerModel
from darts.metrics import wmape,mse, mae, smape,r2_score,rmse,mae
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
from sklearn.model_selection import TimeSeriesSplit

def objective(trial):

    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 168, step=12)
    hidden_size = trial.suggest_int("hidden_size", 32, 256, step=8)
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.05)
    batch_size = trial.suggest_int("batch_size", 64, 256, step=8)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)


    ff_size = trial.suggest_int('ff_size', hidden_size, hidden_size * 4)
    num_blocks = trial.suggest_int('num_blocks', 1, 4)
    activation = trial.suggest_categorical('activation', ['ReLU', 'GELU'])
    norm_type = trial.suggest_categorical('norm_type', ['LayerNormNoBias', 'LayerNorm', 'TimeBatchNorm2d'])
    normalize_before = trial.suggest_categorical('normalize_before', [True, False])

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }

    n_splits = 3
    tscv = TimeSeriesSplit(n_splits=n_splits)
    overall_smape_list = []
    metrics_per_fold = []
    fold = 0

    for train_index, val_index in tscv.split(df):
        print(f"\nFold {fold+1}/{n_splits}")
        train_df_fold = df.iloc[train_index]
        val_df_fold = df.iloc[val_index]
        train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
        val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

        scaler = Scaler()
        scaler = scaler.fit(train_fold)
        train_scaled = scaler.transform(train_fold)
        val_scaled = scaler.transform(val_fold)

        opt_kwargs = {"lr": lr, "weight_decay": weight_decay}

        model = TSMixerModel(
            model_name=f"model_{fold+1}",
            work_dir="/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs",
            input_chunk_length=input_chunk_length,
            output_chunk_length=24,
            hidden_size=hidden_size,
            n_epochs=30,
            batch_size=batch_size,
            dropout=dropout,
            ff_size=ff_size,
            num_blocks=num_blocks,
            activation=activation,
            norm_type=norm_type,
            normalize_before=normalize_before,
            pl_trainer_kwargs=pl_trainer_kwargs,
            loss_fn=torch.nn.L1Loss(),
            optimizer_cls=torch.optim.Adam,
            lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 7, "min_lr": 1e-7},
            save_checkpoints=True,
            force_reset=True,
            random_state=42,
            optimizer_kwargs=opt_kwargs,
        )
        model.fit(
            series=train_scaled,
            val_series=val_scaled,
            verbose=False,
            dataloader_kwargs={"num_workers": 11},
        )

        try:
            loaded_model = model.load_from_checkpoint(f"/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/model_{fold+1}", best=True)
            print(f"Model loaded from checkpoint for trial {trial.number}, fold {fold}")
        except FileNotFoundError:
            print(f"Checkpoint not found. Using the in-memory trained model.")
            loaded_model = model

        forecasts = loaded_model.historical_forecasts(
            val_scaled,
            forecast_horizon=24,
            stride=1,
            retrain=False,
            verbose=False
        )


        forecasts_t = scaler.inverse_transform(forecasts)
        s = scaler.inverse_transform(val_scaled)

        overall_smape_val = smape(s, forecasts_t)
        print(f"SMAPE fold {fold}: {overall_smape_val}")
        overall_smape_list.append(overall_smape_val)
        
        metrics = {}
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
                'WAPE': wmape(s[target], forecasts_t[target]),
            }
        metrics_per_fold.append(metrics)
        fold += 1

    mean_smape = np.mean(overall_smape_list)
    print(f"Média dos SMAPE nos folds: {mean_smape}")

    trial_dict = {
        "trial_number": trial.number,
        "fold_smape": overall_smape_list,
        "mean_smape": mean_smape,
        "hyperparameters": trial.params,
        "metrics_per_fold": metrics_per_fold
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return mean_smape if not np.isnan(mean_smape) else float("inf")

In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 11
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-06-10 18:38:46,624] A new study created in memory with name: no-name-fe5d796b-94b9-43c8-8dd2-b0821f3ce245



Fold 1/3


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Metric val_loss improved. New best score: 8.328
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.326
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.323
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.321
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.319
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.317
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.314
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.312
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.310
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.307
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.305
Metr

Model loaded from checkpoint for trial 0, fold 0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Current value: 7.8277964602582415, Current params: {'input_chunk_length': 60, 'hidden_size': 112, 'dropout': 0.45000000000000007, 'batch_size': 152, 'lr': 0.003207892458697525, 'weight_decay': 0.003018920874829146, 'ff_size': 344, 'num_blocks': 4, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False}
Best value: 75.31818223557848, Best params: {'input_chunk_length': 72, 'hidden_size': 64, 'dropout': 0.4, 'batch_size': 104, 'lr': 0.0034622439183209846, 'weight_decay': 0.0003418366393068562, 'ff_size': 231, 'num_blocks': 1, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False}

Fold 1/3


[I 2025-06-10 19:56:39,162] Trial 75 pruned. Trial was pruned at epoch 0.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current value: 8.072611173525416, Current params: {'input_chunk_length': 108, 'hidden_size': 136, 'dropout': 0.5, 'batch_size': 136, 'lr': 0.0023503995430751434, 'weight_decay': 0.009842414341596325, 'ff_size': 350, 'num_blocks': 4, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False}
Best value: 75.31818223557848, Best params: {'input_chunk_length': 72, 'hidden_size': 64, 'dropout': 0.4, 'batch_size': 104, 'lr': 0.0034622439183209846, 'weight_decay': 0.0003418366393068562, 'ff_size': 231, 'num_blocks': 1, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False}

Fold 1/3


[I 2025-06-10 19:56:41,258] Trial 76 pruned. Trial was pruned at epoch 0.


Current value: 7.971757777532168, Current params: {'input_chunk_length': 72, 'hidden_size': 96, 'dropout': 0.25, 'batch_size': 64, 'lr': 0.0017580463833802421, 'weight_decay': 0.00562229005272647, 'ff_size': 249, 'num_blocks': 4, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False}
Best value: 75.31818223557848, Best params: {'input_chunk_length': 72, 'hidden_size': 64, 'dropout': 0.4, 'batch_size': 104, 'lr': 0.0034622439183209846, 'weight_decay': 0.0003418366393068562, 'ff_size': 231, 'num_blocks': 1, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False}


In [8]:
print(f"Best MASE Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best MASE Value (Minimum): 75.31818223557848
Best Parameters: {'input_chunk_length': 72, 'hidden_size': 64, 'dropout': 0.4, 'batch_size': 104, 'lr': 0.0034622439183209846, 'weight_decay': 0.0003418366393068562, 'ff_size': 231, 'num_blocks': 1, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
